<a href="https://colab.research.google.com/github/dineshkumarDE/pythonprojects/blob/main/sparkRDDoperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from pyspark.sql import SparkSession

spark =SparkSession.builder.appName("RDD operations").getOrCreate()

In [72]:
# prompt: create a variable data containing 10 records with columns customer id ,customer name,city,state,order date,active - indian data with duplicate for two records,have aheader with column names and for now all columns will be inside a single string comma separated and active field is True or False

data = ["customer_id,customer_name,city,state,order_date,active",
    "1,Amit Kumar,Delhi,Delhi,2023-10-26,True",
    "2,Priya Sharma,Mumbai,Maharashtra,2023-10-27,False",
    "3,Rahul Verma,Bangalore,Karnataka,2023-10-28,True",
    "4,Neha Singh,Chennai,Tamil Nadu,2023-10-29,True",
    "5,Vikram Patel,Hyderabad,Telangana,2023-10-30,False",
    "6,Anjali Gupta,Kolkata,West Bengal,2023-10-31,True",
    "7,Rajesh Yadav,Pune,Maharashtra,2023-11-01,False",
    "2,Priya Sharma,Mumbai,Maharashtra,2023-10-27,False", # Duplicate
    "8,Deepika Reddy,Ahmedabad,Gujarat,2023-11-02,True",
    "3,Rahul Verma,Bangalore,Karnataka,2023-10-28,True" # Duplicate
]
rdd=spark.sparkContext.parallelize(data)
rdd.collect()

['customer_id,customer_name,city,state,order_date,active',
 '1,Amit Kumar,Delhi,Delhi,2023-10-26,True',
 '2,Priya Sharma,Mumbai,Maharashtra,2023-10-27,False',
 '3,Rahul Verma,Bangalore,Karnataka,2023-10-28,True',
 '4,Neha Singh,Chennai,Tamil Nadu,2023-10-29,True',
 '5,Vikram Patel,Hyderabad,Telangana,2023-10-30,False',
 '6,Anjali Gupta,Kolkata,West Bengal,2023-10-31,True',
 '7,Rajesh Yadav,Pune,Maharashtra,2023-11-01,False',
 '2,Priya Sharma,Mumbai,Maharashtra,2023-10-27,False',
 '8,Deepika Reddy,Ahmedabad,Gujarat,2023-11-02,True',
 '3,Rahul Verma,Bangalore,Karnataka,2023-10-28,True']

In [73]:
rdd.collect()
rdd.getNumPartitions()



2

In [74]:
spark.sparkContext.defaultParallelism

2

In [75]:
spark.sparkContext.defaultMinPartitions

2

filter header

In [76]:
header = rdd.first()
header

'customer_id,customer_name,city,state,order_date,active'

In [77]:
data_rdd=rdd.filter(lambda row:row!=header)
data_rdd.collect()

['1,Amit Kumar,Delhi,Delhi,2023-10-26,True',
 '2,Priya Sharma,Mumbai,Maharashtra,2023-10-27,False',
 '3,Rahul Verma,Bangalore,Karnataka,2023-10-28,True',
 '4,Neha Singh,Chennai,Tamil Nadu,2023-10-29,True',
 '5,Vikram Patel,Hyderabad,Telangana,2023-10-30,False',
 '6,Anjali Gupta,Kolkata,West Bengal,2023-10-31,True',
 '7,Rajesh Yadav,Pune,Maharashtra,2023-11-01,False',
 '2,Priya Sharma,Mumbai,Maharashtra,2023-10-27,False',
 '8,Deepika Reddy,Ahmedabad,Gujarat,2023-11-02,True',
 '3,Rahul Verma,Bangalore,Karnataka,2023-10-28,True']

use map to split the columns and make first column as integer datatype and last as boolean


In [78]:
def parsed_row(row):
  fields=row.split(",")
  return(
      int(fields[0]),
      fields[1],
      fields[2],
      fields[3],
      fields[4],
      fields[5]=="True"
  )

In [79]:
parsed_data_rdd = data_rdd.map(parsed_row)

In [80]:
parsed_data_rdd.collect()

[(1, 'Amit Kumar', 'Delhi', 'Delhi', '2023-10-26', True),
 (2, 'Priya Sharma', 'Mumbai', 'Maharashtra', '2023-10-27', False),
 (3, 'Rahul Verma', 'Bangalore', 'Karnataka', '2023-10-28', True),
 (4, 'Neha Singh', 'Chennai', 'Tamil Nadu', '2023-10-29', True),
 (5, 'Vikram Patel', 'Hyderabad', 'Telangana', '2023-10-30', False),
 (6, 'Anjali Gupta', 'Kolkata', 'West Bengal', '2023-10-31', True),
 (7, 'Rajesh Yadav', 'Pune', 'Maharashtra', '2023-11-01', False),
 (2, 'Priya Sharma', 'Mumbai', 'Maharashtra', '2023-10-27', False),
 (8, 'Deepika Reddy', 'Ahmedabad', 'Gujarat', '2023-11-02', True),
 (3, 'Rahul Verma', 'Bangalore', 'Karnataka', '2023-10-28', True)]

Advanced RDD operations


In [81]:
#extract a field with Map city and name
name_city_rdd = parsed_data_rdd.map(lambda row:(row[1],row[2]))
name_city_rdd.collect()

[('Amit Kumar', 'Delhi'),
 ('Priya Sharma', 'Mumbai'),
 ('Rahul Verma', 'Bangalore'),
 ('Neha Singh', 'Chennai'),
 ('Vikram Patel', 'Hyderabad'),
 ('Anjali Gupta', 'Kolkata'),
 ('Rajesh Yadav', 'Pune'),
 ('Priya Sharma', 'Mumbai'),
 ('Deepika Reddy', 'Ahmedabad'),
 ('Rahul Verma', 'Bangalore')]

In [82]:
#filter active customers
active_customers_rdd=parsed_data_rdd.filter(lambda x:x[5])
active_customers_rdd.collect()

[(1, 'Amit Kumar', 'Delhi', 'Delhi', '2023-10-26', True),
 (3, 'Rahul Verma', 'Bangalore', 'Karnataka', '2023-10-28', True),
 (4, 'Neha Singh', 'Chennai', 'Tamil Nadu', '2023-10-29', True),
 (6, 'Anjali Gupta', 'Kolkata', 'West Bengal', '2023-10-31', True),
 (8, 'Deepika Reddy', 'Ahmedabad', 'Gujarat', '2023-11-02', True),
 (3, 'Rahul Verma', 'Bangalore', 'Karnataka', '2023-10-28', True)]

In [83]:
#distinct cities

cities =parsed_data_rdd.map(lambda row:row[2]).distinct()
cities.collect()


['Delhi',
 'Mumbai',
 'Hyderabad',
 'Pune',
 'Bangalore',
 'Chennai',
 'Kolkata',
 'Ahmedabad']

In [84]:
#take
parsed_data_rdd.take(1)


[(1, 'Amit Kumar', 'Delhi', 'Delhi', '2023-10-26', True)]

In [85]:
#reduce by key - transformation
combined_rdd = parsed_data_rdd.map(lambda row:(row[2],1)).reduceByKey(lambda x,y:x+y)
combined_rdd.collect()

[('Delhi', 1),
 ('Mumbai', 2),
 ('Hyderabad', 1),
 ('Pune', 1),
 ('Bangalore', 2),
 ('Chennai', 1),
 ('Kolkata', 1),
 ('Ahmedabad', 1)]

In [86]:
#countbyvalue
combined_count = parsed_data_rdd.map(lambda row:row[2]).countByValue()
combined_count

defaultdict(int,
            {'Delhi': 1,
             'Mumbai': 2,
             'Bangalore': 2,
             'Chennai': 1,
             'Hyderabad': 1,
             'Kolkata': 1,
             'Pune': 1,
             'Ahmedabad': 1})

In [87]:
#cities with active customers

active_customers_city_rdd = parsed_data_rdd.filter(lambda row:row[5]).map(lambda row :row[2]).distinct()
active_customers_city_rdd.collect()

['Delhi', 'Bangalore', 'Chennai', 'Kolkata', 'Ahmedabad']

In [88]:
#count active costumers per state
parsed_data_rdd.filter(lambda row:row[5]).map(lambda row:row[3]).countByValue()

defaultdict(int,
            {'Delhi': 1,
             'Karnataka': 2,
             'Tamil Nadu': 1,
             'West Bengal': 1,
             'Gujarat': 1})

In [89]:
spark.stop()